In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random

In [2]:
data = np.load('seed_457.npz')

In [6]:
bot_grid_array = data['bot_grid']
rat_grid_array = data['rat_grid']
time_rem_array = data['time_step_remaining']

In [8]:
tensor_bot = torch.from_numpy(bot_grid_array)
tensor_rat = torch.from_numpy(rat_grid_array)